<a href="https://colab.research.google.com/github/MosaicGroupCMU/African-MRS-Tutorials/blob/main/Google-Colab/Quantum_Espresso_H2O.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to geometry optimization of molecular water using Quantum ESPRESSO

Contributors: [Seda Oturak](https://github.com/sedaoturak), [Ismaila Dabo](https://scholar.google.com/citations?user=rN299m0AAAAJ&hl=en), [Jessica Wen](https://github.com/JessicaWen-PhD), [Cierra Chandler](https://github.com/Cierra-Chandler), [Henry Eya](https://github.com/Henrynweya)



# Step 1: Install libraries, environment, and Quantum ESPRESSO

This part installs libraries for numerical calculations and plotting.

In [2]:
# load plotting libraries
import matplotlib.pyplot as plt

# load numerical libraries
import numpy as np

Quantum ESPRESSO is a plane wave code, which uses Fourier transforms to solve equations in plane wave space. This part installs libraries for fast Fourier transforms (FFTs).

In [3]:
# eliminate text output during installation
%%capture

# install mathematical libraries to peform fast Fourier transforms
# (the exclamation mark means that the command is run under Linux)
! apt-get install -y libfftw3-3 libfftw3-dev libfftw3-doc

The Atomic Simulation Environment (ASE) is a set of tools for running, visualizing, and analyzing simulations. This part installs ASE.

In [4]:
# eliminate text output during installation
%%capture

# install the Atomic simulation environment
# ! apt install ase
! pip install git+https://gitlab.com/ase/ase

Compiling Quantum ESPRESSO from scratch would take a long time. This part uploads pre-compiled executable files (`.x` extension) and additional files containing the pseudopotentials.

In [5]:
# eliminate text output during installation
%%capture

# navigate to main directory named '/content/'
%cd /content/

# download the pre-compiled files in compressed format (under Linux)
#! wget 'https://docs.google.com/uc?export=download&id=1kw_CJMjP6ggDZXDNp5phAqCPpoe2WXCA' -O qe-lite.tgz
!gdown 'https://drive.google.com/uc?export=download&id=13l-Kiyg-F6aYb5lF8M3RsE1hSnLRdGna' -O qe-lite.tgz

# unpack the compressed files (under Linux)
! tar -xvzf qe-lite.tgz

# clean up some files
! rm -rf sample_data qe-lite.tgz

# Step 2: Prepare Quantum ESPRESSO input file for SCF

The calculation is for a water molecule. The definition of the input parameters of the `pw.x` executable can be found at `www.quantum-espresso.org/Doc/INPUT_PW.html`.

Things to note for molecules:
1. The molecule should be put in a vacuum
2. KPOINTS of 1 1 1 is used

In [46]:
# create calculation folder and navigate into it
%mkdir -p /content/water
%cd /content/water/

# create input and write it into the file h20.scf.in
qe_input = """
&control
  prefix='water',
  pseudo_dir = '/content/qe-lite/pseudopotentials',
  outdir='/content/water'
/
&system
  ibrav = 0,
  nat = 3,
  ntyp = 2,
  ecutwfc = 30.0,
  ecutrho = 240.0,
  occupations  = 'smearing'
  degauss      = 0.001
  smearing     = 'marzari-vanderbilt'
/
&electrons
  conv_thr = 1e-8,
/
ATOMIC_SPECIES
O 15.999 O.upf
H 1.008 H.upf

K_POINTS automatic
1 1 1 0 0 0

CELL_PARAMETERS angstrom
10.00000000000000 0.00000000000000 0.00000000000000
0.00000000000000 10.00000000000000 0.00000000000000
0.00000000000000 0.00000000000000 10.00000000000000

ATOMIC_POSITIONS angstrom
O 0.0000000000 0.0000000000 0.2981545000
H 0.0000000000 0.7632390000 -0.2981545000
H 0.0000000000 -0.7632390000 -0.2981545000

"""

with open("h20.scf.in", "w") as f:
    f.write(qe_input)

# print the content of the input file (under Linux)
! cat h20.scf.in

/content/water

&control
  prefix='water',
  pseudo_dir = '/content/qe-lite/pseudopotentials',
  outdir='/content/water'
/
&system
  ibrav = 0,
  nat = 3,
  ntyp = 2,
  ecutwfc = 30.0,
  ecutrho = 240.0,
  occupations  = 'smearing'
  degauss      = 0.001
  smearing     = 'marzari-vanderbilt'
/
&electrons
  conv_thr = 1e-8,
/
ATOMIC_SPECIES
O 15.999 O.upf
H 1.008 H.upf

K_POINTS automatic
1 1 1 0 0 0

CELL_PARAMETERS angstrom
10.00000000000000 0.00000000000000 0.00000000000000
0.00000000000000 10.00000000000000 0.00000000000000
0.00000000000000 0.00000000000000 10.00000000000000

ATOMIC_POSITIONS angstrom
O 0.0000000000 0.0000000000 0.2981545000
H 0.0000000000 0.7632390000 -0.2981545000
H 0.0000000000 -0.7632390000 -0.2981545000



Use ASE tools to extract information from Quantum ESPRESSO input and visualize the crystal.

In [47]:
import ase.io.espresso
from ase import Atoms
from ase.visualize import view
from ase.build import make_supercell
from ase.build import bulk

# extract unit cell information from input file using ASE
input_file = ase.io.espresso.read_espresso_in('h20.scf.in')
water = Atoms(input_file)

# create a supercell (2 × 2 × 2) using ASE
#multiplier = np.identity(3) * 3
#water_supercell = make_supercell(water,multiplier)

"""
#For non-square array (for example 1 x 2 x 1), you can use the follwoing lines

multiplier = np.array([
    [3, 0, 0],  # x-dimension unchanged
    [0, 2, 0],  # y-dimension scaled by 2
    [0, 0, 2]   # z-dimension unchanged
])
water_supercell = make_supercell(water,multiplier)
"""
# visualize the supercell
view(water, viewer='x3d')
#view(water_supercell, viewer='x3d')

#Center the molecule

In [44]:
# Modify the ATOMIC_POSITIONS section to add 5 to each coordinate
lines = qe_input.splitlines()
modified_lines = []
inside_atomic_positions = False

for line in lines:
    if line.strip().startswith("ATOMIC_POSITIONS"):
        inside_atomic_positions = True
        modified_lines.append(line)
    elif inside_atomic_positions:
        if line.strip() == "":  # Exit the section on blank line
            inside_atomic_positions = False
            modified_lines.append(line)
        else:
            parts = line.split()
            if len(parts) == 4:  # Ensure it has atomic symbol + 3 coordinates
                symbol = parts[0]
                coords = [float(c) + 5 for c in parts[1:]]
                modified_lines.append(f"{symbol} {coords[0]:.10f} {coords[1]:.10f} {coords[2]:.10f}")
            else:
                modified_lines.append(line)
    else:
        modified_lines.append(line)

# Join modified lines back into a single string
qe_input_modified = "\n".join(modified_lines)

# Write the modified input into the file h20.scf.in
with open("h20.scf.in", "w") as file:
    file.write(qe_input_modified)

# Print the modified content (optional, for verification)
#print(h20_scf_modified)
!cat h20.scf.in


&control
  prefix='water',
  pseudo_dir = '/content/qe-lite/pseudopotentials',
  outdir='/content/water'
/
&system
  ibrav = 0,
  nat = 3,
  ntyp = 2,
  ecutwfc = 30.0,
  ecutrho = 240.0,
  occupations  = 'smearing'
  degauss      = 0.001
  smearing     = 'marzari-vanderbilt'
/
&electrons
  conv_thr = 1e-8,
/
ATOMIC_SPECIES
O 15.999 O.upf
H 1.008 H.upf

K_POINTS automatic
1 1 1 0 0 0

CELL_PARAMETERS angstrom
10.00000000000000 0.00000000000000 0.00000000000000
0.00000000000000 10.00000000000000 0.00000000000000
0.00000000000000 0.00000000000000 10.00000000000000

ATOMIC_POSITIONS angstrom
O 5.0000000000 5.0000000000 5.2981545000
H 5.0000000000 5.7632390000 4.7018455000
H 5.0000000000 4.2367610000 4.7018455000


# Step 3: Run Quantum ESPRESSO using input file

In [48]:
# run the pw.x executable using si.scf.in to create si.scf.out
! /content/qe-lite/bin/pw.x < h20.scf.in > h20.scf.out

# print the content of the output file (under Linux)
! cat h20.scf.out


     Program PWSCF v.7.4 starts on 14Dec2024 at 10:40:10 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Serial version
     9298 MiB available memory on the printing compute node when the environment starts

     Waiting for input...
     Reading input from standard input

     Current dimensions of program PWSCF are:
     Max number of different atomic species (ntypx) = 10
     Max number of k-points (npk) =  40000
     Max angular momentum in pseudopotentials (lmaxx) =  4
     file O.upf: wavefunction(s)  2P renormalized
    

#Step 4: Extract the total energy of the system

In [49]:
# First method: xtract total energies in rydberg during the self-consistent-field calculation (under Linux)
! grep -e "total energy  " -e "estimated" h20.scf.out #NOTE: grep is a power comand that can seacrh and list the specified term in a file

     total energy              =     -43.80322895 Ry
     estimated scf accuracy    <       0.40856669 Ry
     total energy              =     -43.89585675 Ry
     estimated scf accuracy    <       0.37041931 Ry
     total energy              =     -43.96993004 Ry
     estimated scf accuracy    <       0.00424009 Ry
     total energy              =     -43.97070127 Ry
     estimated scf accuracy    <       0.00075070 Ry
     total energy              =     -43.97066512 Ry
     estimated scf accuracy    <       0.00012596 Ry
     total energy              =     -43.97069060 Ry
     estimated scf accuracy    <       0.00000195 Ry
     total energy              =     -43.97069075 Ry
     estimated scf accuracy    <       0.00000027 Ry
     total energy              =     -43.97069071 Ry
     estimated scf accuracy    <       0.00000012 Ry
!    total energy              =     -43.97069073 Ry
     estimated scf accuracy    <          2.8E-10 Ry


In [50]:
# define physical constants for unit conversion
from scipy.constants import physical_constants
ha_in_ev = physical_constants["Hartree energy in eV"][0]
ry_in_ev = ha_in_ev / 2.

# second method: extract total energy at the end of the self-consistent calculation (using ASE)
output = ase.io.read("/content/water/h20.scf.out")
total_energy = output.get_total_energy()
print("Energy = %.8f eV " % total_energy)
print("Energy = %.8f Ry " % ( total_energy / ry_in_ev ) )

Energy = -598.25167214 eV 
Energy = -43.97068688 Ry 
